# Step 1: Create amplitude model

TensorWaves requires you to first formulate a mathematical {class}`.Model` that you want to fit to your data set. When this model is an amplitude model, it is most convenient to construct it with the {mod}`expertsystem`.

This notebook briefly illustrates how to create such an amplitude model with the {mod}`expertsystem` and how to write it to a recipe file that can be understood by {mod}`tensorwaves`. For more control, have a look at {doc}`the usage guides of the PWA Expert System <expertsystem:usage>`.

In this example, we use the helicity formalism, but you can also use `formalism_type="canonical-helicity"`. As you can see, we analyze the decay $J/\psi \to \pi^0\pi^0\gamma$ here.

```{admonition} Simplified model: $J/\psi \to f_0\gamma$
---
class: dropdown
---
As {doc}`step3` serves to illustrate usage only, we make the amplitude model here a bit simpler by not allowing $\omega$ resonances (which are narrow and therefore hard to fit). For this reason, we can also limit the {class}`~expertsystem.reaction.default_settings.InteractionTypes` to {attr}`~expertsystem.reaction.default_settings.InteractionTypes.STRONG`.
```

In [ ]:
import expertsystem as es

result = es.generate_transitions(
    initial_state=("J/psi(1S)", [-1, +1]),
    final_state=["gamma", "pi0", "pi0"],
    allowed_intermediate_particles=["f(0)"],
    allowed_interaction_types=["strong", "EM"],
)

As a small goodie, you can use [`graphviz`](https://pypi.org/project/graphviz) to {doc}`visualize <expertsystem:usage/visualization>` the generated graphs:

In [ ]:
from graphviz import Source

dot = es.io.asdot(result, collapse_graphs=True)
Source(dot)

Next we convert the {attr}`~expertsystem.reaction.Result.transitions` into an amplitude model (here: {class}`~expertsystem.amplitude.helicity.HelicityModel`). This can be done with {func}`~expertsystem.amplitude.get_builder` and {meth}`~expertsystem.amplitude.helicity.HelicityAmplitudeBuilder.generate`.

In [ ]:
model_builder = es.amplitude.get_builder(result)
model = model_builder.generate()
display(*model.parameters)

The heart of the model is a sympy expression that contains the full description of the intensity model. Note two things:
1. The coefficients for the different amplitudes are **complex** valued.
2. By default there is no dynamics in the model, so it still has to be specified.

We choose to use {func}`~expertsystem.amplitude.dynamics.lineshape.relativistic_breit_wigner_with_ff` as the lineshape for all resonances and use a {class}`~expertsystem.amplitude.dynamics.lineshape.BlattWeisskopf` form factor factor (no dynamics) for the production decay. The {meth}`~expertsystem.amplitude.helicity.HelicityAmplitudeBuilder.set_dynamics` is a convenience interface for replacing the dynamics for intermediate states.

In [ ]:
from expertsystem.amplitude.dynamics.builder import (
    create_non_dynamic_with_ff,
    create_relativistic_breit_wigner_with_ff,
)

model_builder.set_dynamics("J/psi(1S)", create_non_dynamic_with_ff)
for name in result.get_intermediate_particles().names:
    model_builder.set_dynamics(name, create_relativistic_breit_wigner_with_ff)
model = model_builder.generate()

Now let's take another look at the parameters of the model to see which new parameters are there:

In [ ]:
sorted(model.parameters, key=lambda s: s.name)

Finally, we can write the {class}`~expertsystem.amplitude.helicity.HelicityModel` to disk via {mod}`pickle`:

In [ ]:
import pickle

with open("helicity_model.pickle", "wb") as stream:
    pickle.dump(model, stream)

Cool, that's it! We now have a template for an amplitude model with which to {doc}`generate data <step2>` and {doc}`perform a fit <step3>`. In the next steps, we will use use this {class}`~expertsystem.amplitude.helicity.HelicityModel` as a fit model template for {mod}`tensorwaves`.